Imports.

In [43]:
import pandas as pd
from math import floor

import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable

import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import normalize

Calling this row Jupyter Notebook shows us all rows.

In [2]:
pd.options.display.max_rows = 1000

Some reason GPU is not working. I think problem is that then I should move all things to GPU.

In [3]:
# Device configuration
device = torch.device('cpu')#cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cpu')

Import data.

In [5]:
data = pd.read_csv('train.csv')

Split data to y and x.

In [6]:
sale_price = data['SalePrice']
data = data.drop(['SalePrice'],axis=1)
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


Analyzing data.

In [7]:
data.isnull().sum() # Check number of NaN s

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinSF1          0
BsmtFinType2       38
BsmtFinSF2          0
BsmtUnfSF           0
TotalBsmtSF         0
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
1stFlrSF            0
2ndFlrSF            0
LowQualFin

In [8]:
data.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
BsmtExposure      object
BsmtFinType1      object
BsmtFinSF1         int64
BsmtFinType2      object
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
Heating           object


In [9]:
data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [10]:
cat_names = [i for i,v in (data.dtypes == object).items() if v]
cat_names

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [11]:
cont_names = [i for i,v in (data.dtypes == object).items() if not v]
cont_names

['Id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [12]:
len(cat_names),len(cont_names)

(43, 37)

Change strings to categorical data.

In [13]:
data.dtypes.head()

Id               int64
MSSubClass       int64
MSZoning        object
LotFrontage    float64
LotArea          int64
dtype: object

In [14]:
for n in cat_names:
    data[n] = data[n].astype('category').cat.as_ordered()

In [15]:
data.dtypes.head()

Id                int64
MSSubClass        int64
MSZoning       category
LotFrontage     float64
LotArea           int64
dtype: object

In [16]:
data['MSZoning'].cat.categories

Index(['C (all)', 'FV', 'RH', 'RL', 'RM'], dtype='object')

In [17]:
data['MSZoning'].cat.codes

0       3
1       3
2       3
3       3
4       3
5       3
6       3
7       3
8       4
9       3
10      3
11      3
12      3
13      3
14      3
15      4
16      3
17      3
18      3
19      3
20      3
21      4
22      3
23      4
24      3
25      3
26      3
27      3
28      3
29      4
30      0
31      3
32      3
33      3
34      3
35      3
36      3
37      3
38      3
39      3
40      3
41      3
42      3
43      3
44      3
45      3
46      3
47      1
48      4
49      3
50      3
51      4
52      4
53      3
54      3
55      3
56      1
57      3
58      3
59      3
60      3
61      4
62      3
63      4
64      3
65      3
66      3
67      3
68      4
69      3
70      3
71      3
72      3
73      3
74      4
75      4
76      3
77      4
78      3
79      4
80      3
81      4
82      3
83      3
84      3
85      3
86      3
87      1
88      0
89      3
90      3
91      3
92      3
93      0
94      3
95      3
96      3
97      3
98      3
99      3


Fill all NaNs with median when continues data.

In [18]:
for n in cont_names:
    if pd.isnull(data[n]).sum():
        filler = data[n].median()
        data[n] = data[n].fillna(filler)

Turn dtata into torch format. We take log of sale price because evaluation of competition demand us to do so.

In [19]:
sale_price = torch.tensor(np.log(sale_price).values)

In [20]:
sale_price

tensor([12.2477, 12.1090, 12.3172,  ..., 12.4931, 11.8645, 11.9016],
       dtype=torch.float64)

In [21]:
if cat_names and len(cat_names) >= 1: # categorical data
    cats = np.stack([c.cat.codes.values for n,c in data[cat_names].items()], 1) + 1

In [22]:
cats.shape

(1460, 43)

In [23]:
cats = torch.LongTensor(cats.astype(np.int64))

In [24]:
cats

tensor([[4, 2, 0,  ..., 0, 9, 5],
        [4, 2, 0,  ..., 0, 9, 5],
        [4, 2, 0,  ..., 0, 9, 5],
        ...,
        [4, 2, 0,  ..., 3, 9, 5],
        [4, 2, 0,  ..., 0, 9, 5],
        [4, 2, 0,  ..., 0, 9, 5]])

Normalizing data.

In [25]:
if cont_names and len(cont_names) >= 1:
    conts = np.stack([c.astype('float32').values for n,c in data[cont_names].items()], 1)
    means, stds = (conts.mean(0), conts.std(0))
    conts = (conts - means[None]) / stds[None]
    stats = means,stds

In [26]:
conts.shape

(1460, 37)

In [27]:
conts = torch.FloatTensor(conts)

In [28]:
conts

tensor([[-1.7309,  0.0734, -0.2209,  ..., -0.0877, -1.5991,  0.1387],
        [-1.7285, -0.8726,  0.4603,  ..., -0.0877, -0.4891, -0.6145],
        [-1.7261,  0.0734, -0.0846,  ..., -0.0877,  0.9909,  0.1387],
        ...,
        [ 1.7261,  0.3099, -0.1755,  ...,  4.9531, -0.4891,  1.6452],
        [ 1.7285, -0.8726, -0.0846,  ..., -0.0877, -0.8591,  1.6452],
        [ 1.7309, -0.8726,  0.2333,  ..., -0.0877, -0.1191,  0.1387]])

In [29]:
cat_szs = [len(data[n].cat.categories)+1 for n in cat_names]
emb_szs = [(c, min(50, (c+1)//2)) for c in cat_szs]
print(emb_szs)

[(6, 3), (3, 2), (3, 2), (5, 3), (5, 3), (3, 2), (6, 3), (4, 2), (26, 13), (10, 5), (9, 5), (6, 3), (9, 5), (7, 4), (9, 5), (16, 8), (17, 9), (5, 3), (5, 3), (6, 3), (7, 4), (5, 3), (5, 3), (5, 3), (7, 4), (7, 4), (7, 4), (6, 3), (3, 2), (6, 3), (5, 3), (8, 4), (6, 3), (7, 4), (4, 2), (6, 3), (6, 3), (4, 2), (4, 2), (5, 3), (5, 3), (10, 5), (7, 4)]


In [30]:
x_data_conts = conts
x_data_cats  = cats
y_data = sale_price

In [31]:
x_data_conts.shape

torch.Size([1460, 37])

In [32]:
x_data_cats.shape

torch.Size([1460, 43])

In [33]:
y_data.shape

torch.Size([1460])

And then we split data into train, valid and test sets.

In [34]:
train_size = 0.8
valid_size = 0.25

train_cnt = floor(x_data_cats.shape[0] * train_size)

x_train_cats  = x_data_cats[0:train_cnt]
x_train_conts = x_data_conts[0:train_cnt]
y_train = y_data[0:train_cnt]

valid_cnt = floor((x_data_cats.shape[0] - train_cnt) * valid_size)

x_valid_cats  = x_data_cats[train_cnt:train_cnt+valid_cnt]
x_valid_conts = x_data_conts[train_cnt:train_cnt+valid_cnt]
y_valid = y_data[train_cnt:train_cnt+valid_cnt]

x_test_cats  = x_data_cats[train_cnt+valid_cnt:]
x_test_conts = x_data_conts[train_cnt+valid_cnt:]
y_test = y_data[train_cnt+valid_cnt:]

print("x_train_cats:",x_train_cats.shape)
print("x_train_conts:",x_train_conts.shape)
print("y_train:",y_train.shape)
print("x_valid_cats:",x_valid_cats.shape)
print("x_valid_conts:",x_valid_conts.shape)
print("y_valid:",y_valid.shape)
print("x_test_cats:",x_test_cats.shape)
print("x_test_conts:",x_test_conts.shape)
print("y_test:",y_test.shape)

x_train_cats: torch.Size([1168, 43])
x_train_conts: torch.Size([1168, 37])
y_train: torch.Size([1168])
x_valid_cats: torch.Size([73, 43])
x_valid_conts: torch.Size([73, 37])
y_valid: torch.Size([73])
x_test_cats: torch.Size([219, 43])
x_test_conts: torch.Size([219, 37])
y_test: torch.Size([219])


In [35]:
# Network Parameters
learning_rate = 1e-2
epochs = 500
batch_size = 64
num_input = x_data_conts.shape[1]
num_classes = 1
num_hidden_1 = 200
num_hidden_2 = 200
num_hidden_3 = 100
num_hidden_4 = 100

In [36]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes, emb_szs, n_cont):
        super().__init__()
        
        self.embeds = nn.ModuleList([nn.Embedding(ne,de) for ne,de in emb_szs])
        n_emb = sum(e.embedding_dim for e in self.embeds)
        self.n_emb,self.n_cont = n_emb,n_cont
        
        
        self.layer1 = nn.Sequential(
            nn.Linear(self.n_emb+self.n_cont,num_hidden_1),
            nn.ReLU(),
            nn.BatchNorm1d(num_hidden_1),
            nn.Dropout())
        self.layer2 = nn.Sequential(
            nn.Linear(num_hidden_1,num_hidden_2),
            nn.ReLU(),
            nn.BatchNorm1d(num_hidden_2),
            nn.Dropout())
        self.layer3 = nn.Sequential(
            nn.Linear(num_hidden_2,num_hidden_3),
            nn.ReLU(),
            nn.BatchNorm1d(num_hidden_3),
            nn.Dropout())
        self.layer4 = nn.Sequential(
            nn.Linear(num_hidden_3,num_hidden_4),
            nn.ReLU(),
            nn.BatchNorm1d(num_hidden_4),
            nn.Dropout())
        self.layer5 = nn.Sequential(
            nn.Linear(num_hidden_4,num_classes))
        
    def forward(self, x_cat, x_cont):
        x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]
        x = torch.cat(x, 1)
        x = torch.cat([x, x_cont], 1)
        
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        return out

In [37]:
model = NeuralNetwork(num_classes,emb_szs=emb_szs,n_cont=len(cont_names)).to(device)

In [38]:
train_data = torch.utils.data.TensorDataset(x_train_conts,x_train_cats,y_train)
train_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=False)

valid_data = torch.utils.data.TensorDataset(x_valid_conts,x_valid_cats,y_valid)
valid_loader = torch.utils.data.DataLoader(valid_data,batch_size=batch_size,shuffle=False)

test_data = torch.utils.data.TensorDataset(x_test_conts,x_test_cats,y_test)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=batch_size,shuffle=False)

In [39]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [40]:
costs = []
lrn_rate_sizes = []

In [41]:
# Train the model
total_step = int(len(x_train_conts) / batch_size)
for epoch in range(epochs):
    for i, (x_conts,x_cats,y) in enumerate(train_loader):
        # Forward pass
        outputs = model(x_cats,x_conts)
        loss = criterion(outputs, y[:,None].float())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i) % 100 == 0:
            for i2, (x2_conts,x2_cats,y2) in enumerate(valid_loader):
                loss2 = criterion(outputs, y2[:,None].float())
                break
            print ('Epoch [{}/{}], Step [{}/{}], Train_Loss: {:.4f}, Valid_Loss: {:.4f}' 
                   .format(epoch+1, epochs, i+1, total_step, loss.item(), loss2.item()))
print("done!")

Epoch [1/500], Step [1/18], Train_Loss: 142.9024, Valid_Loss: 143.3448
Epoch [2/500], Step [1/18], Train_Loss: 74.9722, Valid_Loss: 75.5546
Epoch [3/500], Step [1/18], Train_Loss: 4.1440, Valid_Loss: 4.3436
Epoch [4/500], Step [1/18], Train_Loss: 2.7536, Valid_Loss: 3.0897
Epoch [5/500], Step [1/18], Train_Loss: 2.5735, Valid_Loss: 2.5031
Epoch [6/500], Step [1/18], Train_Loss: 2.4933, Valid_Loss: 2.2625
Epoch [7/500], Step [1/18], Train_Loss: 2.2255, Valid_Loss: 2.6660
Epoch [8/500], Step [1/18], Train_Loss: 2.1227, Valid_Loss: 2.2382
Epoch [9/500], Step [1/18], Train_Loss: 2.0156, Valid_Loss: 2.4030
Epoch [10/500], Step [1/18], Train_Loss: 1.4478, Valid_Loss: 1.6835
Epoch [11/500], Step [1/18], Train_Loss: 2.0659, Valid_Loss: 2.4636
Epoch [12/500], Step [1/18], Train_Loss: 1.3461, Valid_Loss: 1.7174
Epoch [13/500], Step [1/18], Train_Loss: 1.5762, Valid_Loss: 1.6120
Epoch [14/500], Step [1/18], Train_Loss: 1.3575, Valid_Loss: 1.6430
Epoch [15/500], Step [1/18], Train_Loss: 1.7657, Va

Epoch [123/500], Step [1/18], Train_Loss: 0.9359, Valid_Loss: 1.1462
Epoch [124/500], Step [1/18], Train_Loss: 1.1123, Valid_Loss: 1.3895
Epoch [125/500], Step [1/18], Train_Loss: 0.8048, Valid_Loss: 1.2964
Epoch [126/500], Step [1/18], Train_Loss: 1.1161, Valid_Loss: 1.3777
Epoch [127/500], Step [1/18], Train_Loss: 1.0360, Valid_Loss: 1.1782
Epoch [128/500], Step [1/18], Train_Loss: 0.9543, Valid_Loss: 1.3801
Epoch [129/500], Step [1/18], Train_Loss: 1.0009, Valid_Loss: 1.3306
Epoch [130/500], Step [1/18], Train_Loss: 0.7179, Valid_Loss: 1.1020
Epoch [131/500], Step [1/18], Train_Loss: 0.8475, Valid_Loss: 1.1381
Epoch [132/500], Step [1/18], Train_Loss: 0.7518, Valid_Loss: 0.8494
Epoch [133/500], Step [1/18], Train_Loss: 0.7189, Valid_Loss: 0.9718
Epoch [134/500], Step [1/18], Train_Loss: 0.8670, Valid_Loss: 1.1537
Epoch [135/500], Step [1/18], Train_Loss: 0.6132, Valid_Loss: 0.9571
Epoch [136/500], Step [1/18], Train_Loss: 1.0271, Valid_Loss: 1.3703
Epoch [137/500], Step [1/18], Trai

Epoch [242/500], Step [1/18], Train_Loss: 0.3975, Valid_Loss: 0.6996
Epoch [243/500], Step [1/18], Train_Loss: 0.2359, Valid_Loss: 0.4711
Epoch [244/500], Step [1/18], Train_Loss: 0.3501, Valid_Loss: 0.6716
Epoch [245/500], Step [1/18], Train_Loss: 0.2575, Valid_Loss: 0.4919
Epoch [246/500], Step [1/18], Train_Loss: 0.3289, Valid_Loss: 0.5457
Epoch [247/500], Step [1/18], Train_Loss: 0.2436, Valid_Loss: 0.5504
Epoch [248/500], Step [1/18], Train_Loss: 0.2833, Valid_Loss: 0.4561
Epoch [249/500], Step [1/18], Train_Loss: 0.2941, Valid_Loss: 0.4964
Epoch [250/500], Step [1/18], Train_Loss: 0.2270, Valid_Loss: 0.3971
Epoch [251/500], Step [1/18], Train_Loss: 0.2965, Valid_Loss: 0.5358
Epoch [252/500], Step [1/18], Train_Loss: 0.1902, Valid_Loss: 0.4176
Epoch [253/500], Step [1/18], Train_Loss: 0.2751, Valid_Loss: 0.5074
Epoch [254/500], Step [1/18], Train_Loss: 0.2240, Valid_Loss: 0.5642
Epoch [255/500], Step [1/18], Train_Loss: 0.2296, Valid_Loss: 0.4254
Epoch [256/500], Step [1/18], Trai

Epoch [361/500], Step [1/18], Train_Loss: 0.0389, Valid_Loss: 0.2834
Epoch [362/500], Step [1/18], Train_Loss: 0.0358, Valid_Loss: 0.2711
Epoch [363/500], Step [1/18], Train_Loss: 0.0293, Valid_Loss: 0.2703
Epoch [364/500], Step [1/18], Train_Loss: 0.0342, Valid_Loss: 0.2968
Epoch [365/500], Step [1/18], Train_Loss: 0.0314, Valid_Loss: 0.3301
Epoch [366/500], Step [1/18], Train_Loss: 0.0326, Valid_Loss: 0.2988
Epoch [367/500], Step [1/18], Train_Loss: 0.0322, Valid_Loss: 0.2659
Epoch [368/500], Step [1/18], Train_Loss: 0.0321, Valid_Loss: 0.2978
Epoch [369/500], Step [1/18], Train_Loss: 0.0327, Valid_Loss: 0.2766
Epoch [370/500], Step [1/18], Train_Loss: 0.0296, Valid_Loss: 0.2894
Epoch [371/500], Step [1/18], Train_Loss: 0.0253, Valid_Loss: 0.2806
Epoch [372/500], Step [1/18], Train_Loss: 0.0223, Valid_Loss: 0.2808
Epoch [373/500], Step [1/18], Train_Loss: 0.0324, Valid_Loss: 0.2757
Epoch [374/500], Step [1/18], Train_Loss: 0.0266, Valid_Loss: 0.2778
Epoch [375/500], Step [1/18], Trai

Epoch [481/500], Step [1/18], Train_Loss: 0.0103, Valid_Loss: 0.2983
Epoch [482/500], Step [1/18], Train_Loss: 0.0141, Valid_Loss: 0.3091
Epoch [483/500], Step [1/18], Train_Loss: 0.0108, Valid_Loss: 0.3084
Epoch [484/500], Step [1/18], Train_Loss: 0.0179, Valid_Loss: 0.2985
Epoch [485/500], Step [1/18], Train_Loss: 0.0116, Valid_Loss: 0.2905
Epoch [486/500], Step [1/18], Train_Loss: 0.0139, Valid_Loss: 0.3000
Epoch [487/500], Step [1/18], Train_Loss: 0.0119, Valid_Loss: 0.3123
Epoch [488/500], Step [1/18], Train_Loss: 0.0100, Valid_Loss: 0.3030
Epoch [489/500], Step [1/18], Train_Loss: 0.0118, Valid_Loss: 0.2926
Epoch [490/500], Step [1/18], Train_Loss: 0.0133, Valid_Loss: 0.2852
Epoch [491/500], Step [1/18], Train_Loss: 0.0088, Valid_Loss: 0.2854
Epoch [492/500], Step [1/18], Train_Loss: 0.0180, Valid_Loss: 0.3190
Epoch [493/500], Step [1/18], Train_Loss: 0.0133, Valid_Loss: 0.2711
Epoch [494/500], Step [1/18], Train_Loss: 0.0086, Valid_Loss: 0.3080
Epoch [495/500], Step [1/18], Trai

For some reason this is not even close what I got from Kaggle.

In [42]:
# Test the model
model.eval()
total = 0
nums = 0
with torch.no_grad():
    for x_conts, x_cats, y in test_loader:
        outputs = model(x_cats,x_conts)
        
        y = y[:,None].float().reshape((-1,1))
        loss = criterion(outputs, y.float())
        nums += 1
        total += loss

    print('Loss:',total/nums)

Loss: tensor(0.1172)
